In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
metadata = []
for dirname, _, filenames in os.walk('/kaggle/input/amazon-bucket'):
    for filename in filenames:
        metadata.append(os.path.join(dirname, filename))

In [ ]:
imagedata = []
for dirname, _, filenames in os.walk('/kaggle/input/amazonimage'):
    for filename in filenames:
        imagedata.append(os.path.join(dirname, filename))

In [ ]:
import json
with open(metadata[0], "r") as f:
    print(json.load(f))

In [ ]:
import skimage
from skimage.io import imread, imshow
image = imread(imagedata[0],as_gray = True)
imshow(image)

In [ ]:
m1 = []
im1 = []
for meta in metadata:
    meta = meta.replace("/kaggle/input/amazon-bucket/METADATA/","")
    meta = meta.replace(".json","")
    m1.append(meta)

In [ ]:
for img in imagedata:
    img = img.replace("/kaggle/input/amazonimage/IMAGEDATA/","")
    img = img.replace(".jpg","")
    im1.append(img)

In [ ]:
m11 = set(m1)
intersect = m11.intersection(im1)
intersect = list(intersect)

In [ ]:
newmetadata = []
newimagedata = []
for inter in intersect:
    newmetadata.append("/kaggle/input/amazon-bucket/METADATA/"+inter+".json")
    newimagedata.append("/kaggle/input/amazonimage/IMAGEDATA/"+inter+".jpg")

In [ ]:
metafiles = []
for i,file in enumerate(newmetadata):
    if i%25==0:
        with open(file, "r") as f:
            metafiles.append(json.load(f))

In [ ]:
quantity = []
keepos = []
def get_total_quantity(metafiles):
    number = []
    for i,meta in enumerate(metafiles):
        if meta['EXPECTED_QUANTITY']<13:
            number.append(meta['EXPECTED_QUANTITY'])
            keepos.append(i)
    return number

In [ ]:
quantity = get_total_quantity(metafiles)
print(len(quantity))

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import CondensedNearestNeighbour
from matplotlib import pyplot
from numpy import where

In [ ]:
import random

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from skimage.filters import prewitt_h,prewitt_v
from skimage.feature import hog

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
 
class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
     
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [ ]:
from PIL import Image
import pickle
import bz2
import _pickle as cPickle
imagefiles = []
jj = 0
for i,img in enumerate(newimagedata):
    if i%25==0:
        if jj in keepos:
            img1 = Image.open(img).convert("RGB")
            img1 = img1.resize((180,180),Image.BILINEAR)
            #150,150
            img1.save("Image1.jpg")
            #imagefiles.append(imread("Image1.jpg",as_gray = True))
            image = imread("Image1.jpg")
            #edges_prewitt_vertical = prewitt_v(image)
            imagefiles.append(image)
        jj = jj+1)

In [ ]:
from sklearn.model_selection import train_test_split
image_train, image_test, quan_train, quan_test = train_test_split(imagefiles, quantity, test_size=0.25, random_state=42,stratify=quantity)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
import skimage
 
# create an instance of each transformer
grayify = RGB2GrayTransformer()
hogify = HogTransformer(
    pixels_per_cell=(14, 14), 
    cells_per_block=(2,2), 
    orientations=9, 
    block_norm='L2-Hys'
)
scalify = StandardScaler()
 
# call fit_transform on each transform converting X_train step by step
X_train_gray = grayify.fit_transform(image_train)
X_train_hog = hogify.fit_transform(X_train_gray)
X_train_prepared = scalify.fit_transform(X_train_hog)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.metrics import f1_score,confusion_matrix,ConfusionMatrixDisplay

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X_test_gray = grayify.transform(image_test)
X_test_hog = hogify.transform(X_test_gray)
X_test_prepared = scalify.transform(X_test_hog)

In [ ]:
def loosen_error(predictions, real_values, thresh = 0):
    """A loosen error definition. If the model predicts a near value (the difference of the prediction and real label
    is at most 1), then a credit is given to the model. """
    correct = 0
    somewhat_correct = 0
    total = 0
    for i in range(len(real_values)):
        total += 1
        if real_values[i] >= 2 or predictions[i] >= 2: 
            if real_values[i] - predictions[i] == 0:
                correct += 1
                somewhat_correct += 1
            if abs(real_values[i] - predictions[i]) <= 1:
                if thresh == 0:
                    somewhat_correct += 1 - 1/real_values[i]
                else:
                    somewhat_correct += thresh
    return somewhat_correct/total, correct/total

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import RidgeClassifier,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#gnb = RandomForestClassifier(n_estimators=500)
gnb = SGDClassifier(random_state=42, max_iter=2000, tol=1e-3)
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))
print(loosen_error(predictions,quan_test))

In [ ]:
gnb = RandomForestClassifier(n_estimators=500)
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))
print(loosen_error(predictions,quan_test))

In [ ]:
gnb = MLPClassifier(solver='sgd',max_iter=2000)
gnb.fit(X_train_prepared,quan_train)
predictions = gnb.predict(X_test_prepared)
print(f1_score(quan_test,predictions,average='macro'))
fig,ax = plt.subplots(figsize=(20, 10))
cm = confusion_matrix(quan_test,predictions,labels=np.unique(quantity))
disp = ConfusionMatrixDisplay(cm, display_labels=np.unique(quantity))
disp.plot(ax=ax)
plt.show()
print(gnb.score(X_test_prepared, quan_test))
print(loosen_error(predictions,quan_test))